<a href="https://colab.research.google.com/github/mimikuo365/COVID19-Infection-and-Mortality-Prediction/blob/main/dimensionality_reduction_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Run with GPU

- [Scikit-learn Tutorial – Beginner’s Guide to GPU Accelerating ML Pipelines
](https://developer.nvidia.com/blog/scikit-learn-tutorial-beginners-guide-to-gpu-accelerating-ml-pipelines/)

- [Code](https://colab.research.google.com/drive/1rY7Ln6rEE1pOlfSHCYOVaqt8OvDO35J0#forceEdit=true&sandboxMode=true&scrollTo=dCE8WhO3HpL_)

In [ ]:
!nvidia-smi

Mon Nov  8 05:27:21 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8    28W / 149W |      3MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# This get the RAPIDS-Colab install files and test check your GPU.  Run this and the next cell only.
# Please read the output of this cell.  If your Colab Instance is not RAPIDS compatible, it will warn you and give you remediation steps.
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/env-check.py

fatal: destination path 'rapidsai-csp-utils' already exists and is not an empty directory.
Traceback (most recent call last):
  File "rapidsai-csp-utils/colab/env-check.py", line 24, in <module>
    Please use 'Runtime -> Factory Reset Runtimes...', which will allocate you a different GPU instance, to try again."""
Exception: 
                  Unfortunately Colab didn't give you a RAPIDS compatible GPU (P4, P100, T4, or V100), but gave you a Tesla K80.

                  Please use 'Runtime -> Factory Reset Runtimes...', which will allocate you a different GPU instance, to try again.


In [ ]:
# This will update the Colab environment and restart the kernel.  Don't run the next cell until you see the session crash.
!bash rapidsai-csp-utils/colab/update_gcc.sh
import os
os._exit(00)

Updating your Colab environment.  This will restart your kernel.  Don't Panic!
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ub

In [ ]:
# This will install CondaColab.  This will restart your kernel one last time.  Run this cell by itself and only run the next cell once you see the session crash.
import condacolab
condacolab.install()

In [ ]:
# you can now run the rest of the cells as normal
import condacolab
condacolab.check()

In [ ]:
# Installing RAPIDS is now 'python rapidsai-csp-utils/colab/install_rapids.py <release> <packages>'
# The <release> options are 'stable' and 'nightly'.  Leaving it blank or adding any other words will default to stable.
# The <packages> option are default blank or 'core'.  By default, we install RAPIDSAI and BlazingSQL.  The 'core' option will install only RAPIDSAI and not include BlazingSQL, 
!python rapidsai-csp-utils/colab/install_rapids.py stable
import os
os.environ['NUMBAPRO_NVVM'] = '/usr/local/cuda/nvvm/lib64/libnvvm.so'
os.environ['NUMBAPRO_LIBDEVICE'] = '/usr/local/cuda/nvvm/libdevice/'
os.environ['CONDA_PREFIX'] = '/usr/local'

In [ ]:
import cuml as sklearn

/usr/local/lib/python3.7/site-packages/cudf/utils/gpu_utils.py:93: UserWarning: You will need a GPU with NVIDIA Pascal™ or newer architecture
Detected GPU 0: Tesla K80 
Detected Compute Capability: 3.7
  f"You will need a GPU with NVIDIA Pascal™ or "


# Setup Environment and Load Functions

## Load Libraries

In [ ]:
!pip install scikit-learn==0.24

     |████████████████████████████████| 22.3 MB 2.0 MB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [ ]:
# sklearn library
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
from sklearn.decomposition import PCA
from sklearn.manifold import LocallyLinearEmbedding
from sklearn.metrics import mean_absolute_percentage_error
from sklearn import linear_model  
from sklearn.svm import SVR

In [ ]:
from random import randint, randrange
from skimage import io
from skimage.transform import resize

import cv2
import os
import pandas as pd
import numpy as np
import pickle
import plotly.express as px
import matplotlib.pyplot as plt

In [ ]:
!pip install epiweeks
from epiweeks import Week, Year
from datetime import date

## Pickle Function

In [ ]:
def loadPickle(path):
  file = open(path, "rb")
  output = pickle.load(file)
  return output

In [ ]:
def saveAsPickle(path, dictionary_data):
  file = open(path, "wb")
  pickle.dump(dictionary_data, file)
  file.close()

In [ ]:
def loadDataFromPickle(path):
  # Load pickle file from folder
  info_dict = loadPickle(path)
  return info_dict

## Math Util

In [ ]:
def getEpiWeek(origin_str):
  date_ls = origin_str.split('-')
  return Week.fromdate(date(int(date_ls[0]), int(date_ls[1]), int(date_ls[2])))

## Dataframe

In [ ]:
def loadStructuredData(csv_path):
  df = pd.DataFrame()
  if os.path.isdir(csv_path):
    for filename in os.listdir(csv_path):
      file_path = os.path.join(csv_path, filename)
      df = df.append(pd.read_csv(file_path))
  elif os.path.isfile(csv_path):
    df = pd.read_csv(csv_path)
  else:
    print('Error: Not folder or file')
  return df

## Image


In [ ]:
def plotData(x_asix, y_axis, title):
  plt.figure(figsize = (12, 8))
  plt.plot(x_axis, y_axis, color ='black')
  plt.ylabel('Dengue Cases')
  plt.title(title)
  plt.tick_params(axis='both',labelsize=3)
  plt.setp(plt.gca().get_xticklabels(), rotation=30, horizontalalignment='right')

  plt.show()

In [ ]:
def readImg(img_path, resize_ratio=None):
  img = io.imread(img_path)
  print(os.path.basename(img_path))
  print('    origin shape:'.ljust(10), img.shape)

  if resize_ratio:
    new_shape = (x / resize_ratio for x in img.shape)
    img = resize(img, resize_ratio)
  print('    resized shape:'.ljust(10), img.shape)
  return img

## Obtain both structural data and images

In [ ]:
def combineData(img_folder, df, resize_ratio=None):
  info_dict = {'LastDayWeek':[], 'cases_medellin':[], 'Image':[]}
  img_list = os.listdir(img_folder)

  for index, row in df.iterrows():
    name = row['LastDayWeek']
    week_df = str(getEpiWeek(name))
    case = row['cases_medellin']
    print(name)
    for img_name in img_list:
      '''
      Join the items in the list with a empty string '' : https://www.w3schools.com/python/ref_string_join.asp
      If input is [2016, -, 10, -, 31]; then returns 2016-10-31
      check: https://stackoverflow.com/questions/13174468/how-do-you-join-all-items-in-a-list/13175535
      '''
      new_img_name = ''.join(i for i in img_name if i.isdigit() or i == '-')
      week_img = str(getEpiWeek(new_img_name))

      if week_df == week_img:
        img_path = os.path.join(img_folder, img_name)
        img = readImg(img_path, resize_ratio)

        info_dict['Image'].append(img)
        info_dict['LastDayWeek'].append(name)
        info_dict['cases_medellin'].append(case)
        break

  return info_dict

# Match Images and Cases

In [ ]:
# Load data from one of the source
def loadData(csv_folder, img_folder, option=None, resize_ratio=None):
  if option is None:
    # Get data by combining from csv and images
    df = loadStructuredData(csv_folder)
    info_dict = combineData(img_folder, df, resize_ratio)
    print(len(info_dict['LastDayWeek']), len(info_dict['Image']), len(info_dict['cases_medellin']))

    # Save dictionary into pickle
    pickle_folder = '/content/drive/MyDrive/Dengue forecasting with Satellite Images/pickle'
    pickle_name = '12_band.pkl'
    path = os.path.join(pickle_folder, pickle_name)
    saveAsPickle(path, info_dict)

  else:
    # Load data from previous pickle file
    info_dict = loadDataFromPickle(option)
  return info_dict

In [ ]:
# Define main folder
dataset_folder = '/content/drive/MyDrive/Dengue forecasting with Satellite Images/dataset'

# Define image folder
image_name = 'image/image_12_band'
img_folder = os.path.join(dataset_folder, image_name)

# Define csv folder
csv_name = 'csv/merge_cases_timeseries.csv'
csv_folder = os.path.join(dataset_folder, csv_name)

pickle_folder = '/content/drive/MyDrive/Dengue forecasting with Satellite Images/pickle'
pickle_name = '12_band.pkl'
path = os.path.join(pickle_folder, pickle_name)
info_dict = loadData(csv_folder, img_folder, None, 0.5)

print('INFO_DICT'.center(50, '-'))
print('keys:', info_dict.keys())
print('')

print('DENGUE CASES'.center(50, '-'))
print('Max weekly dengue cases:', max(info_dict['cases_medellin']))
print('Min weekly dengue cases:', min(info_dict['cases_medellin']))
print('')

print('WEEKS'.center(50, '-'))
print('Max week:', max(info_dict['LastDayWeek']))
print('Min week:', min(info_dict['LastDayWeek']))

2007-01-06
2007-01-13
2007-01-20
2007-01-27
2007-02-03
2007-02-10
2007-02-17
2007-02-24
2007-03-03
2007-03-10
2007-03-17
2007-03-24
2007-03-31
2007-04-07
2007-04-14
2007-04-21
2007-04-28
2007-05-05
2007-05-12
2007-05-19
2007-05-26
2007-06-02
2007-06-09
2007-06-16
2007-06-23
2007-06-30
2007-07-07
2007-07-14
2007-07-21
2007-07-28
2007-08-04
2007-08-11
2007-08-18
2007-08-25
2007-09-01
2007-09-08
2007-09-15
2007-09-22
2007-09-29
2007-10-06
2007-10-13
2007-10-20
2007-10-27
2007-11-03
2007-11-10
2007-11-17
2007-11-24
2007-12-01
2007-12-08
2007-12-15
2007-12-22
2007-12-29
2008-01-05
2008-01-12
2008-01-19
2008-01-26
2008-02-02
2008-02-09
2008-02-16
2008-02-23
2008-03-01
2008-03-08
2008-03-15
2008-03-22
2008-03-29
2008-04-05
2008-04-12
2008-04-19
2008-04-26
2008-05-03
2008-05-10
2008-05-17
2008-05-24
2008-05-31
2008-06-07
2008-06-14
2008-06-21
2008-06-28
2008-07-05
2008-07-12
2008-07-19
2008-07-26
2008-08-02
2008-08-09
2008-08-16
2008-08-23
2008-08-30
2008-09-06
2008-09-13
2008-09-20
2008-09-27

TypeError: ignored

# Data Distrubtion Visualization

In [ ]:
x_axis = info_dict['LastDayWeek']
y_axis = info_dict['cases_medellin']
plotData(x_axis, y_axis, 'Images & Cases Intersection')

dataset_folder = '/content/drive/MyDrive/Dengue forecasting with Satellite Images/dataset'
csv_name = 'csv/merge_cases_timeseries.csv'
csv_folder = os.path.join(dataset_folder, csv_name)
df = loadStructuredData(csv_folder)

x_axis = df['LastDayWeek']
y_axis = df['cases_medellin'].astype(int)
plotData(x_axis, y_axis, 'Cases')

# Split Training and Testing Sets

In [ ]:
# Split the data into training (0.8) and testing (0.2)
train_val_ratio = 0.8
train_num = int(len(info_dict['Image']) * train_val_ratio)

# Change list to array
origin_dimension_X = np.array(info_dict['Image'])
labels = np.array(info_dict['cases_medellin'])

# Reshape image from (1205, 765, 12) to (11061900, 1)
X = origin_dimension_X.reshape(len(info_dict['Image']), -1)
print('Image reshaped from', origin_dimension_X[0].shape, 'to', X[0].shape)
print(''.center(60,'-'))

origin_X_train = X[:train_num]
y_train = labels[:train_num]
origin_X_test = X[train_num:]
y_test = labels[train_num:]

print('Total number of weeks:'.ljust(30), len(X), 'weeks')
print('Training input:'.ljust(30), origin_X_train.shape)
print('Training output:'.ljust(30), y_train.shape)
print('Testing input:'.ljust(30), origin_X_test.shape)
print('Testing output:'.ljust(30), y_test.shape) 

Image reshaped from (1205, 765, 12) to (11061900,)
------------------------------------------------------------
Total number of weeks:         112 weeks
Training input:                (89, 11061900)
Training output:               (89,)
Testing input:                 (23, 11061900)
Testing output:                (23,)


# Dimensionality Reduction

## PCA

In [ ]:
def calCorrelation(data, label):
  corr = np.corrcoef(data, label)[0, 1]
  return round(corr, 4)

In [ ]:
def dimension_reduct_with_PCA(origin_X_train, origin_X_test, y_train, load_from_pickle=False, pickle_name=None):
  origin_X_train = np.float32(origin_X_train)
  origin_X_test = np.float32(origin_X_test)
  y_train = np.float32(y_train)

  if load_from_pickle:
    # Load PCA model
    path = os.path.join(pickle_folder, pickle_name)
    pca = loadPickle(path)
    pca_X_train = pca.transform(origin_X_train)

  else:
    # Create PCA object
    # Select the number of components such that the amount of variance that needs to be explained is greater than the percentage specified by n_components
    print('train start')
    pca = PCA(n_components=0.95) 
    pca_X_train = pca.fit_transform(origin_X_train)
    print('train end')

    # Save PCA model
    if pickle_name:
      path = os.path.join(pickle_folder, pickle_name)
      saveAsPickle(path, pca)

  pca_X_test = pca.transform(origin_X_test)
  print(origin_X_train.shape)
  print(pca_X_train.shape)  

  return pca, pca_X_train, pca_X_test

In [ ]:
def plotDistribution(corr_list, X_train, y_train):
  sorted_corr_list = list(corr_list)
  sorted_corr_list.sort(reverse=True)

  max_index = corr_list.index(sorted_corr_list[0])
  sec_max_index = corr_list.index(sorted_corr_list[1])

  color_list = []
  max_cases = max(y_train)

  for i in y_train:
    color = [i / max_cases, 0, 0]
    color_list.append(color)

  color_list = np.array(color_list)

  plt.subplot(1, 2, 1)
  plt.scatter(X_train[:,max_index], X_train[:,sec_max_index], c=color_list)
  plt.subplot(1, 2, 2)
  plt.scatter(np.full(y_train.shape, 0, dtype=int), y_train, c=color_list)
  plt.tight_layout()
  plt.show()

## LLE

In [ ]:
def dimension_reduct_with_LLE(origin_X_train, origin_X_test, y_train):
  embedding = LocallyLinearEmbedding(n_components=2, neighbors_algorithm='kd_tree')
  lle_X_train = embedding.fit_transform(origin_X_train, y_train)
  lle_X_test = embedding.transform(origin_X_test)
  return lle_X_train, lle_X_test

# Metric

In [ ]:
# Mean Absolute Percentage Error
def get_MAPE_score(y_true, y_pred):
    # The lower the MAPE the better
    # TODO: Consult Dr. Septarshi about zero case (y_true) 

    y_true, y_pred = np.array(y_true), np.array(y_pred)
    print(np.mean(np.abs((y_true - y_pred) / y_true)) * 100)
    print(mean_absolute_percentage_error(y_true, y_pred))

    # Example:
    # (0.5 - 0.4) / 0.5 = 0.2
    # (0.5 - 0.5) / 0.5 = 0.0 <-> y_true = y_pred

In [ ]:
def get_MAE_score(y_test, y_pred):
  return round(mean_absolute_error(y_test, y_pred), 5)

# Statistical Models

In [ ]:
def classified_with_LR(X_train, X_test, y_train, y_test):
  regr = linear_model.LinearRegression()
  regr.fit(X_train, y_train)
  y_pred = regr.predict(X_test)
  MAE = get_MAE_score(y_test, y_pred)
  get_MAPE_score((y_test, y_pred))
  return MAE

In [ ]:
def classified_with_SVR(X_train, X_test, y_train, y_test):
  regressor = SVR(C=1.0, epsilon=0.2)
  regressor.fit(X_train, y_train)
  y_pred = regressor.predict(X_test)
  MAE = get_MAE_score(y_test, y_pred)
  get_MAPE_score(y_test, y_pred)
  return MAE

# Test Different Combination

In [ ]:
# PCA
pca, pca_X_train, pca_X_test = dimension_reduct_with_PCA(origin_X_train, origin_X_test, y_train)

# Linear Regression
# MAE = classified_with_LR(pca_X_train, pca_X_test, y_train, y_test)
# print('[PCA][LR] MAE', MAE)

# SVR 
# MAE = classified_with_SVR(pca_X_train, pca_X_test, y_train, y_test)
# print('[PCA][SVR] MAE', MAE)

classified_with_LR(origin_X_train, origin_X_test, y_train, y_test)

train start


In [ ]:
print(pca.components_.shape)

## Principle Component Analysis

In [ ]:
fig = plt.figure(figsize=(8,16)) 
fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.05, wspace=0.05) 

for i in range(len(pca.components_)): 
  ax = fig.add_subplot(len(pca.components_)/5 + 1, 5, i+1, xticks=[], yticks=[]) 
  scaler = MinMaxScaler(feature_range=(0,1))
  rgb_pca = pca.components_[i,:].reshape(1205, 765, 3)
  norm_pc_img = scaler.fit_transform(rgb_pca[:,:,2].reshape(-1, 1))

  ax.imshow(np.reshape(norm_pc_img, (1205, 765)), cmap=plt.cm.seismic, interpolation='nearest')

In [ ]:
# Image1 x PC
fig = plt.figure(figsize=(8,16)) 
fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.05, wspace=0.05) 
img = origin_X_train[0].reshape(1205, 765, 3)

for i in range(len(pca.components_)): 
  ax = fig.add_subplot(len(pca.components_)/5 + 1, 5, i+1, xticks=[], yticks=[]) 
  pc_img = np.multiply(pca.components_[i,:].reshape(1205, 765, 3), img)
  scaler = MinMaxScaler(feature_range=(0,1))
  norm_pc_img = scaler.fit_transform(pc_img.reshape(-1, 1))
  ax.imshow(np.reshape(norm_pc_img, (1205, 765, 3)), cmap=plt.cm.bone, interpolation='nearest')

In [ ]:
fig = plt.figure(figsize=(8,16)) 
fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.05, wspace=0.05) 

# Image shape = (1, 1205 x 765 x 3)
# 51 PC = (1, 1205 x 765 x 3)

for i in range(len(pca.components_)): 
  ax = fig.add_subplot(len(pca.components_)/5 + 1, 5, i+1, xticks=[], yticks=[]) 

  # pca.components_[i,:] = (1205 x 765 x 3, 1) -> (1205, 765, 3)
  rgb_pca = pca.components_[i,:].reshape(1205, 765, 3)
  
  # Normalize values to range 0 ~ 1
  scaler = MinMaxScaler(feature_range=(0,1)) 

  # rgb_pca[:,:,0]: H(row), W(column), RGB Channel
  # rgb_pca[:,:,0] = (1205, 765, 1) -> (1205 x 765, 1)
  # TODO: Check dimension of (rgb_pca[:,:,0].reshape(-1, 1))
  norm_pc_img = scaler.fit_transform(rgb_pca[:,:,0].reshape(-1, 1)) 
  
  # (921825,1) -> (1205, 765)
  ax.imshow(np.reshape(norm_pc_img, (1205, 765)), cmap=plt.cm.seismic, interpolation='nearest')

In [ ]:
fig = plt.figure(figsize=(8,16)) 
fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.05, wspace=0.05) 

for i in range(len(pca.components_)): 
  ax = fig.add_subplot(len(pca.components_)/5 + 1, 5, i+1, xticks=[], yticks=[]) 
  scaler = MinMaxScaler(feature_range=(0,1))
  rgb_pca = pca.components_[i,:].reshape(1205, 765, 3)
  norm_pc_img = scaler.fit_transform(rgb_pca[:,:,1].reshape(-1, 1))
  ax.imshow(np.reshape(norm_pc_img, (1205, 765)), cmap=plt.cm.seismic, interpolation='nearest')

In [ ]:
fig = plt.figure(figsize=(8,16)) 
fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.05, wspace=0.05) 

for i in range(len(pca.components_)): 
  ax = fig.add_subplot(len(pca.components_)/5 + 1, 5, i+1, xticks=[], yticks=[]) 
  scaler = MinMaxScaler(feature_range=(0,1))
  rgb_pca = pca.components_[i,:].reshape(1205, 765, 3)
  norm_pc_img = scaler.fit_transform(rgb_pca[:,:,2].reshape(-1, 1))
  # TODO: choose pc graph with |corr| > 0.2
  # TODO: make a average input image to overlap with PC figure
  # TODO: Ask david about the lines inside images
  ax.imshow(np.reshape(norm_pc_img, (1205, 765)), cmap=plt.cm.seismic, interpolation='nearest')

In [ ]:
lle_X_train, lle_X_test = dimension_reduct_with_LLE(origin_X_train, origin_X_test, y_train)

# Linear Regression
MAE = classified_with_LR(lle_X_train, lle_X_test, y_train, y_test)
print('[LLE][LR] MAE', MAE)

# SVR 
MAE = classified_with_SVR(lle_X_train, lle_X_test, y_train, y_test)
print('[LLE][SVR] MAE', MAE)

In [ ]:
# Save PC to drive
img = origin_X_train[0].reshape(1205, 765, 3)
plt.imshow(img)
img_number_per_row = 3
total_pc = len(pca.components_)
print(total_pc)

for index, pc in enumerate(pca.components_):
  # plt.subplot(total_pc / img_number_per_row + 1, 
  #             img_number_per_row, 
  #             index + 1)
  # pc_img = np.multiply(pc.reshape(1205, 765, 3), img)

  pc_img = pc.reshape(1205, 765, 3)
  scaler = MinMaxScaler(feature_range=(0,1))
  norm_pc_img = scaler.fit_transform(pc_img.reshape(-1, 1)).reshape(1205, 765, 3)
  plt.imshow(norm_pc_img)
  plt.savefig('/content/drive/MyDrive/Dengue forecasting with Satellite Images/pca_components/' + str(index) + '.png')